In [1]:
import os
import shutil
import pandas as pd
import re

In [91]:
def reqno(givenstring):
    req_pattern = re.compile(r'cp.+art.+r.sc.+?\d{5}')
    try:
        match = req_pattern.search(givenstring.lower()).group()
        return re.search(r'\d{5}',match).group()
    except:
        return 'unknown'
        
def place(givenstring):
    station = ['TDN','PLG','VTP','RCC','FAC']
    patterns =  [['tdn.{1,2}st.t.on','th.o.+d.en.+st.t.on','th.o.+di.n.+st.t.on'],
                 ['pl.{1,2}st.t.on','phu.c.+l.ng.+st.t.on'],
                 ['vt.{1,2}st.t.on','v.n.+th.nh.+st.t.on'],
                 ['rc.{1,2}st.t.on','r.ch.+chi.c.+st.t.on','r.ch.+ch.ec.+st.t.on'],
                 ['factory']]
    for i in range(len(station)):
        for pattern in patterns[i]:
            #print(station[i]+pattern)
            try:
                if re.search(pattern,givenstring)!=None:
                    return station[i]
            except:
                pass
    return 'naPlace'

def reqdate(givenstring):
    req_pattern = re.compile(r'.n date.+\d{1,2}.+\d{2,4}')
    try:
        match = req_pattern.search(givenstring).group()
        reqdate = re.findall(r'[\d\w]+',match)
        return '{}-{}-{}'.format(reqdate[-3],reqdate[-2],reqdate[-1])
    except:
        return "naDate"

def workitem(givenstring):
    works = ['brick','platering','metalcladding','ceiling','epoxy','paint','stone',
             'partition','waterproofing','topping']
    patterns =  [
            ['brick','stiffe.+','lintel','tie.+beam'], #brick
            ['plaster.+','render.+'],
            ['m.tal.+w.rk','met.l.+w.rk','m.ta.+cladding','met.l.+clad.+'], #metal
            ['susp.nd.d','suspension','cei.ing','c.i.ing','ceilling'], #ceiling
            ['epoxy','.poxy','ep.xy'], #epoxy
            ['paint','pa.nt','c.ng.+t.c.+s.n'], #paint          
            ['stone.+w.rk','st.ne.+w.rk','sto.e.+w.rk','ston.+w.rk','s.one.+w.rk'], # stone
            ['partition','part.t..n','p..tition','dry wall'], #partition
            ['water.+fing'], #waterproofing
            ['topping','concrete','mortar','to..ing']
            ]

    for i in range(len(works)):
        for pattern in patterns[i]:
            #print(station[i]+pattern)
            try:
                if re.search(pattern,givenstring)!=None:
                    #print(pattern)
                    return works[i]
            except:
                pass
    return 'naWork'

def reqfor(givenstring):

    pattern = r'request for [\W\s.].*?[\n\r]'
    try:
        match = re.search(pattern,givenstring.lower(),flags=re.DOTALL).group()
        return match
    except:
        return "unknown"    

def itpno(givenstring):

    pattern = r'ITP No:.+\d{3}.+\d{5}'
    try:
        match = re.search(pattern,givenstring.lower(),flags=re.DOTALL).group()
        return re.findall(r'[\d\w]+',match)[-1]
    except:
        return "unknown"   
    
def decode_risc(givenstring):
    return r'{0} {1} {2} {3}'.format(
                                     reqno(givenstring),
                                     place(givenstring),
                                     workitem(givenstring),
                                     reqdate(givenstring),
                                     itpno(givenstring)
                                    )
def copyjpg(src_file,dst_file):
    shutil.copy(src_file, dst_file)

df.columns = ['Unn', 'pdf', 'jpg', 'json', 'txt']
del(df['Unn'])
df.to_csv(mappingfname,index=False)

In [93]:
########### CONTROL BOARD ########################################
mappingfnames    = [
    'D:/tmrisc/2021 filemapping.csv',
    'D:/tmrisc/2022 filemapping.csv',
    #'D:/tmrisc/2023 filemapping.csv',
]
       
output_folder   = 'D:/tmrisc/output/'
txt_folder      = 'D:/tmrisc/txt'
#################################################################
for mappingfname in mappingfnames:
    df = pd.read_csv(mappingfname) # read file
    if "output" not in df: df['output'] = pd.NA  # checking available of collumns txt 
    #for index in df['output'].isna().index: 
    for index in df['output'].index: 
        f = open(df.loc[index,'txt'],'r')
        content = f.read().lower()
        df.loc[index,'output']=output_folder + decode_risc(content) + '.jpg'
        #copyjpg(df.loc[index,'jpg'],df.loc[index,'output'])
    df.to_csv(mappingfname,index=False)

In [ ]:
#export to output_jpg file
mappingfname    = 'D:/tmrisc/2022 filemapping.csv'  
df = pd.read_csv(mappingfname) # read file
for index in df['output'].index: 
    copyjpg(df.loc[index,'jpg'],df.loc[index,'output'])

# TESTING

In [89]:
def test(givenstring):    
    works = ['brick','platering','metalcladding','ceiling','epoxy','paint','stone','partition',
             'waterproofing','topping']
    patterns =  [
            ['brick','bl.ck','stiffer','lintel','tie.+beam'], #brick
            ['plaster.+','render.+'],
            ['m.tal.+w.rk','met.l.+w.rk','m.ta.+cladding','met.l.+clad.+'], #metal
            ['susp.nd.d','suspension','cei.ing','c.i.ing','ceilling'], #ceiling
            ['epoxy','.poxy','ep.xy'], #epoxy
            ['paint','pa.nt','c.ng.+t.c.+s.n'], #paint          
            ['stone.+w.rk','st.ne.+w.rk','sto.e.+w.rk','ston.+w.rk','s.one.+w.rk'], # stone
            ['partition','part.t..n','p..tition'], #partition
            ['water.+fing'], #waterproofing
            ['topping','to.+in.']
            ]

    for i in range(len(works)):
        for pattern in patterns[i]:
            #print(station[i]+pattern)
            try:
                if re.search(pattern,givenstring)!=None:
                    print(pattern)
                    return works[i]
            except:
                pass
    return 'naWork'

In [87]:
f = open(r'D:/tmrisc/txt/2021 02305.txt','r')
givenstring = f.read().lower()
test(givenstring)

to.+in.


'topping'

In [88]:
print(givenstring)

ho chi minh city urban railway construction project/ dw ån xåy dwng dw&ng sit dö thi tp.hcm	

ben thanh — suoi tien section (line i) / doan bén thånh — subi tién (tuyén s6 1)	

contract package 2: civil (elevated & de t) / gåi thåu s6 2: xå dvng (doan trén cao vå depot	

employer	employer's representative	

general contractor	

mana ement authori for urban railwa	njpt association for general consultants' semces	

sumitomo-cienco 6 consortium scc	

request for inspection and survey check (risc)	

(phiéu dé nghi giam va kiém tra trac d4c)	

request no:cp2 - art - risc - 28022-a	itp no: hcmc-262-stgn-css-pln-00080-c	

scc's request (dé nghi cüa tong thai.} scc)	

the following works are / will be ready for inspection (cåc cong vi$c sau dåy dä/sé sån sang dé duvc kiém tra)	

work item (heng mvc)	goncrete/ concrete work station	

bb mat nhö	

-frecetve1*	

request for (kiém tra vd)	- final inspection	

-nghiem thu hoån thånh	

location (v/ tri)	phuoc long station - nhä ga phu&c long	

_plat

In [ ]:
r'ITP No:.+\d{3}.+\d{5}'